In [61]:
# Path rephraser 
def path_rephraser(path: str) -> str:
    return path.replace('\\', '/')

#print(rephrased_path)  # Output: C:/Users/jj402/Desktop/ML

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, recall_score, accuracy_score


In [63]:
# Rephrase the file path

path = r"C:\Users\jj402\Desktop\ML\loan_default_ml\data\application_train.csv"
rephrased_path = path_rephraser(path)

# Load the dataset
data = pd.read_csv(rephrased_path)

Day 2's work must lie here. 
* Any feature engineering, creation of the new features must be done before the train-test splits and the statistical trnasformation must be done during the preprocessing. This are the best practices followed in the industry

In [64]:
# Missing values table and the mean 
# Identify the null values in each column, find the mean of the missing null values and convert to percentage and then solve it with worst offenders first.

missing_pct = (data.isnull().mean() * 100).sort_values(ascending=False)
# remove columns with 0% missing values

missing_pct = missing_pct[missing_pct > 0.0]
for idx, val in missing_pct.items():
    print(idx,'==',val)

COMMONAREA_AVG == 69.87229725115525
COMMONAREA_MODE == 69.87229725115525
COMMONAREA_MEDI == 69.87229725115525
NONLIVINGAPARTMENTS_MEDI == 69.43296337366793
NONLIVINGAPARTMENTS_MODE == 69.43296337366793
NONLIVINGAPARTMENTS_AVG == 69.43296337366793
FONDKAPREMONT_MODE == 68.38617155158677
LIVINGAPARTMENTS_AVG == 68.35495315614726
LIVINGAPARTMENTS_MEDI == 68.35495315614726
LIVINGAPARTMENTS_MODE == 68.35495315614726
FLOORSMIN_MODE == 67.84862980511267
FLOORSMIN_AVG == 67.84862980511267
FLOORSMIN_MEDI == 67.84862980511267
YEARS_BUILD_AVG == 66.49778381911543
YEARS_BUILD_MODE == 66.49778381911543
YEARS_BUILD_MEDI == 66.49778381911543
OWN_CAR_AGE == 65.9908100848425
LANDAREA_MEDI == 59.376737742714894
LANDAREA_AVG == 59.376737742714894
LANDAREA_MODE == 59.376737742714894
BASEMENTAREA_MODE == 58.51595552679416
BASEMENTAREA_MEDI == 58.51595552679416
BASEMENTAREA_AVG == 58.51595552679416
EXT_SOURCE_1 == 56.381072546998325
NONLIVINGAREA_MODE == 55.17916432257708
NONLIVINGAREA_AVG == 55.17916432257

Now based on the results of missing data we can find out some details. 

Rule of thumb: 
* 60% missing -> suspicious
* 80% missing -> often useless (but not always)

Therefore with so much missing data does imputing make sense of am I just inventing data ?

***Does missingness mean something ?***

Missingness itself may correlate with default risk but missing != random.

Example: 
* `OWN_CAR_AGE` missing → probably does not own a car
* `OCCUPATION_TYPE` missing → unemployed / informal work
* `EXT_SOURCE_*` missing → no credit bureau data

***Why we do this BEFORE feature engineering***

If you blindly impute:
* Median hides the fact that value was missing
* Model loses signal

Auditing missingness helps you decide:
* Which columns deserve missing flags
* Which ones are just noise

*“Missing values are not always errors; they can encode real-world processes such as ineligibility, absence of records, or socioeconomic differences.”*

| Feature | Description | Why might this be missing ? |
|--------|-------------|------------------------------|
|COMMONAREA_AVG |Normalized information about building where the client lives|A large fraction of housing-related features are missing, likely because detailed building records are unavailable or inapplicable for many clients. This missingness is probably systematic and may encode socioeconomic information.|
|OCCUPATION_TYPE |What kind of occupation does the client have|One plausible reason for missing OCCUPATION_TYPE is that applicants such as pensioners or unemployed individuals may not have a standardized occupation recorded.|
|EXT_SOURCE_3 |Risk score obtained from an external credit-related data source.|Missing values likely indicate absence of records in the external data source, which may correspond to limited or no credit history. This missingness is therefore systematic and potentially informative rather than random.|
|AMT_REQ_CREDIT_BUREAU_HOUR	|Number of enquiries to Credit Bureau about the client one hour before application |Missing values likely correspond to applicants with no available bureau record or inapplicable credit history, making missingness potentially informative.|



In [65]:
# To verify if the missingness in occupation type is correlated to income type (it is)

pd.crosstab(
    data['NAME_INCOME_TYPE'],
    data['OCCUPATION_TYPE'].isnull(),
    normalize='index'
)


OCCUPATION_TYPE,False,True
NAME_INCOME_TYPE,,
Businessman,0.800000,0.200000
Commercial associate,0.828295,0.171705
Maternity leave,0.800000,0.200000
Pensioner,0.000090,0.999910
State servant,0.825508,0.174492
Student,0.722222,0.277778
Unemployed,0.000000,1.000000
Working,0.843047,0.156953


In [66]:
print([col for col in data.columns])

['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELE

In [67]:
# Filtering the columns related to housing details like 'COMMONAREA_AVG' for combining the housing missingness indicator

housing_columns = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']
housing_cols = []

for i in housing_columns: # Removing the subscript 'AVG' in all the columns
    if i.endswith('_AVG'):
        z = i.replace('_AVG','_') 
        housing_cols.append(z)

housing_columns = [ # storing all the housing details related columns with different subscripts like 'AVG', 'MODE', etc
    col for col in data.columns
    if col.startswith(tuple(housing_cols))
]

# Addition of the missingness indicator for all the features related to housing details

data['housing_info_missing'] = (
    data[housing_columns].isnull().all(axis=1)
).astype(int)

#data['housing_info_missing'].value_counts(normalize=True)

C:\Users\jj402\AppData\Local\Temp\ipykernel_5592\868205075.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['housing_info_missing'] = (


In [68]:
# Missingness indicators for external sources
data['EXT_SOURCE_1_missing'] = data['EXT_SOURCE_1'].isnull().astype(int)
data['EXT_SOURCE_3_missing'] = data['EXT_SOURCE_3'].isnull().astype(int)
data['EXT_SOURCE_3_missing'] = data['EXT_SOURCE_3'].isnull().astype(int)

# Credit enquiry details missingness indicators
credit_enquiry_cols = [
    col for col in data.columns
    if col.startswith('AMT_REQ_CREDIT_BUREAU')
]

data['Credit_info_missing'] = (
    data[credit_enquiry_cols].isnull().all(axis=1).astype(int)
)

data['OCCUPATION_TYPE'] = data['OCCUPATION_TYPE'].fillna('Not_Applicable') 
# Not adding a missing flag for this case instead encoding a new category for not applicable for unemployed or pensioners


C:\Users\jj402\AppData\Local\Temp\ipykernel_5592\1098630738.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['EXT_SOURCE_1_missing'] = data['EXT_SOURCE_1'].isnull().astype(int)
C:\Users\jj402\AppData\Local\Temp\ipykernel_5592\1098630738.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['EXT_SOURCE_3_missing'] = data['EXT_SOURCE_3'].isnull().astype(int)
C:\Users\jj402\AppData\Local\Temp\ipykernel_5592\1098630738.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

In [74]:
# New features based on credit ratios 
data['credit_income_ratio'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL']
data['annuity_income_ratio'] = data['AMT_ANNUITY'] / data['AMT_INCOME_TOTAL']
data['employment_age_ratio'] = data['DAYS_EMPLOYED'] / data['DAYS_BIRTH']

data.replace([np.inf, -np.inf], np.nan, inplace=True)

data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,housing_info_missing,EXT_SOURCE_1_missing,EXT_SOURCE_3_missing,Credit_info_missing,credit_income_ratio,annuity_income_ratio,employment_age_ratio
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,0.0,1.0,0,0,0,0,2.007889,0.121978,0.067329
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0,0,1,0,4.790750,0.132217,0.070862
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,1,1,0,0,2.000000,0.100000,0.011814
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,1,1,1,1,2.316167,0.219900,0.159905
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,1,1,1,0,4.222222,0.179963,0.152418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,NaN,NaN,NaN,0,0,1,1,1.617143,0.174971,0.025303
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,NaN,NaN,NaN,0,1,1,1,3.743750,0.166687,-17.580890
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,1.0,0.0,1.0,0,0,0,0,4.429176,0.195941,0.529266
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0.0,0.0,0.0,0,1,0,0,2.164368,0.118158,0.400134


In [78]:
#Splitting the dataset into training and testing sets

y = data['TARGET']
X = data.drop(columns=['TARGET'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state= 42, stratify= y)

if y_train.value_counts(normalize=True).all() == y_test.value_counts(normalize=True).all():
    print("\nBoth the splits are same\n") 
else : 
    print("\nBoth the sets are not same\n")



Both the splits are same



In [79]:
# Splitting the numerical columns and categorical columns separately for imputation 

num_cols = X_train.select_dtypes(include = ['int64','float64']).columns
cat_cols = X_train.select_dtypes(include = ['str']).columns

# Preprocessing Imputer pipelines are defined for numeric and categorical values separately

numeric_pipeline = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

categorical_pipeline = Pipeline( steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

# Definition of the Preprocessor block using the imputer pipelines 

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_pipeline, num_cols),
        ('cat',categorical_pipeline, cat_cols)
    ]
)

In [80]:
# initializing the baseline model

model = Pipeline(steps = [
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Training the model with the train data
model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transform

In [81]:
# Evaluation of the model 

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

roc_auc = roc_auc_score(y_test, y_proba)
recall = recall_score(y_test, y_pred)

print(f"ROC-AUC: {roc_auc:.3f}")
print(f"Recall: {recall:.3f}")

ROC-AUC: 0.750
Recall: 0.010


|Metrics|Day 1 score| Day 2 score|
|-------|-----------|------------|
|ROC-AUC|0.629|0.750|
|Recall|0.000|0.010|

## Day 2 — Feature Engineering Summary

**Goal:** Improve baseline model performance by encoding real-world signal through features.

### What was done
- Audited missingness and identified systematic (non-random) missing values.
- Added missingness indicators for external credit score features (`EXT_SOURCE_1`, `EXT_SOURCE_3`) to preserve absence-of-information.
- Encoded meaningful categorical missingness in `OCCUPATION_TYPE` as `Not_Applicable`.
- Created domain-driven ratio features:
  - `credit_income_ratio`
  - `annuity_income_ratio`
  - `employment_age_ratio`
- Added scaling for numerical features in the preprocessing pipeline.

### Results
| Metric | Day 1 | Day 2 |
|------|------|------|
| ROC-AUC | 0.629 | **0.750** |
| Recall | 0.000 | **0.010** |

### Key Takeaways
- Ratio features significantly improved the model’s ability to rank risk.
- Missingness indicators restored information lost during imputation.
- Feature engineering improved ROC-AUC more than recall, highlighting model and threshold limitations.

**Conclusion:**  
Feature engineering added meaningful signal and substantially improved model performance, validating the importance of domain knowledge in classical ML.
